In [2]:
import operator
import math
import random
import numpy as np
from deap import algorithms, base, creator, tools, gp

def division_operator(numerator, denominator):
    if denominator == 0:
        return 1
    return numerator / denominator

def eval_func(individual, points):
    func = toolbox.compile(expr=individual)
    errors = [(func(x) - 5*x**3 + 6*x**2 - 8*x + 1)**2 for x in points]
    return math.fsum(errors) / len(points),

def create_toolbox():
    pset = gp.PrimitiveSet("MAIN", 1)
    pset.addPrimitive(operator.add, 2)
    pset.addPrimitive(operator.sub, 2)
    pset.addPrimitive(operator.mul, 2)
    pset.addPrimitive(division_operator, 2)
    pset.addPrimitive(operator.neg, 1)
    pset.addPrimitive(math.cos, 1)
    pset.addPrimitive(math.sin, 1)
    pset.addEphemeralConstant("rand101", lambda: random.randint(-1,1))
    pset.renameArguments(ARG0='x')
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)
    toolbox = base.Toolbox()
    toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("compile", gp.compile, pset=pset)
    toolbox.register("evaluate", eval_func, points=[x/10. for x in range(-10,10)])
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("mate", gp.cxOnePoint)
    toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
    toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
    toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
    toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
    return toolbox

if __name__ == "__main__":
    random.seed(7)
    toolbox = create_toolbox()
    population = toolbox.population(n=450)
    hall_of_fame = tools.HallOfFame(1)
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", np.mean)
    mstats.register("std", np.std)
    mstats.register("min", np.min)
    mstats.register("max", np.max)
    probab_crossover = 0.4
    probab_mutate = 0.2
    number_gen = 10
    population, log = algorithms.eaSimple(population, toolbox,
        cxpb=probab_crossover, mutpb=probab_mutate, ngen=number_gen,
        stats=mstats, halloffame=hall_of_fame, verbose=True)


   	      	                    fitness                    	                      size                     
   	      	-----------------------------------------------	-----------------------------------------------
gen	nevals	avg    	gen	max   	min    	nevals	std    	avg    	gen	max	min	nevals	std    
0  	450   	65.3059	0  	106.21	41.3678	450   	10.3394	3.73556	0  	7  	2  	450   	1.62449
1  	238   	62.0487	1  	1216.72	41.8757	238   	55.0655	3.80444	1  	10 	1  	238   	1.77813
2  	232   	55.6696	2  	79.6906	41.5774	232   	7.4513 	3.96889	2  	13 	1  	232   	1.87946
3  	248   	52.9092	3  	88.0317	41.5774	248   	7.83529	4.17333	3  	13 	1  	248   	1.83574
4  	232   	50.9084	4  	104.865	32.714 	232   	9.00519	4.87556	4  	13 	1  	232   	2.12552
5  	208   	47.6437	5  	97.69  	32.714 	208   	7.60911	5.52444	5  	15 	1  	208   	2.4323 
6  	218   	45.7589	6  	149.31 	32.714 	218   	9.39208	6.68667	6  	15 	1  	218   	2.82088
7  	231   	43.4631	7  	104.865	26.5704	231   	9.41258	7.89333	7  	19 	1  	23